In [6]:
EXP_NAME = "e037-ens"

In [1]:
import polars as pl

# CVの計算を行う

In [2]:
cv_fold0 = pl.read_csv(
    "../" + "trained_models/e037-use-all-info/valid_dataset_e037-use-all-info.csv"
)
cv_fold1 = pl.read_csv(
    "../"
    + "trained_models/e037-fold1-use-all-info/valid_dataset_e037-fold1-use-all-info.csv"
)
cv_fold2 = pl.read_csv(
    "../"
    + "trained_models/e037-fold2-use-all-info/valid_dataset_e037-fold2-use-all-info.csv"
)

In [3]:
oof = pl.concat([cv_fold0, cv_fold1, cv_fold2])

In [4]:
oof

index,Clothing ID,Age,Title,Review Text,Rating,label,Positive Feedback Count,Division Name,Department Name,Class Name,fold,valid_pred
i64,i64,i64,str,str,i64,i64,i64,str,str,str,i64,f64
3,0,45,"""""","""I was really pleased with this…",5,1,9,"""General""","""Bottoms""","""Skirts""",0,0.9974885
5,0,62,"""Beautiful day to evening skirt""","""A lovely skirt and i'm so glad…",4,1,0,"""General""","""Bottoms""","""Skirts""",0,0.998319
6,0,73,"""Too small""","""I usually wear a 6 and ordered…",2,0,7,"""General""","""Bottoms""","""Skirts""",0,0.161452
18,2,38,"""Amazing find!""","""I can?t describe my love for t…",5,1,0,"""General""","""Tops""","""Sweaters""",0,0.998862
20,2,42,"""Cropped and boxy""","""It is a beautiful sweater, but…",2,0,0,"""General""","""Tops""","""Sweaters""",0,0.04406
…,…,…,…,…,…,…,…,…,…,…,…,…
9988,232,45,"""Yes!""","""Oh my goodness, i paid full pr…",5,1,0,"""General""","""Dresses""","""Dresses""",2,0.999013
9989,232,48,"""A colorful big box""","""Since this sold out so fast i …",1,0,0,"""General""","""Dresses""","""Dresses""",2,0.029619
9992,232,52,"""Nicely fitted, lined dress but…","""Great dress, nicely structured…",4,1,0,"""General""","""Dresses""","""Dresses""",2,0.959724


In [9]:
import os

os.makedirs("../" + f"trained_models/{EXP_NAME}", exist_ok=True)
oof.write_csv("../" + f"trained_models/{EXP_NAME}/large-oof.csv")

In [10]:
from sklearn.metrics import roc_auc_score

cv_score = roc_auc_score(oof["label"], oof["valid_pred"])
print(f"CV Score: {cv_score}")

CV Score: 0.9696432121740296


In [11]:
pred_fold0 = pl.read_csv(
    "../" + "trained_models/e037-use-all-info/submission_e037-use-all-info_cv0.9756.csv"
)
pred_fold1 = pl.read_csv(
    "../"
    + "trained_models/e037-fold1-use-all-info/submission_e037-fold1-use-all-info_cv0.9681.csv"
)
pred_fold2 = pl.read_csv(
    "../"
    + "trained_models/e037-fold2-use-all-info/submission_e037-fold2-use-all-info_cv0.9697.csv"
)

In [16]:
((pred_fold0 + pred_fold1 + pred_fold2) / 3).write_csv(
    "../" + f"trained_models/{EXP_NAME}/ens37-cv{cv_score:.4f}.csv"
)